## 1. Setup and Configuration <a name="setup"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import sys
import os

# Add parent directory to path
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Import modules
from kerr_utils import (
    setup_prd_style, COLORS,
    horizon_radius, ergosphere_radius, isco_radius
)

from experiments.trajectory_classifier import (
    OrbitProfile, TrajectoryOutcome, ThrustStrategy,
    classify_orbit, compute_orbit_properties,
    compute_key_radii_vs_spin, SPIN_VALUES
)

from experiments.phase_space import (
    plot_orbit_profile_map, plot_periapsis_depth_map,
    plot_effective_potential, plot_spin_comparison
)

from experiments.thrust_comparison import (
    SimulationConfig, compare_strategies,
    simulate_single_impulse, simulate_geodesic
)

from experiments.ensemble import (
    EnsembleConfig, run_ensemble, find_sweet_spot
)

# Setup plotting style
setup_prd_style()
%matplotlib inline

print("Modules loaded successfully!")

In [ ]:
# Configuration
M = 1.0  # Black hole mass (normalized)
a = 0.95  # Default spin

# Key radii
r_plus = horizon_radius(a, M)
r_erg = ergosphere_radius(np.pi/2, a, M)
r_isco = isco_radius(a, M, prograde=True)

print(f"Black hole configuration:")
print(f"  Spin: a/M = {a}")
print(f"  Horizon: r+ = {r_plus:.4f} M")
print(f"  Ergosphere (equator): r_erg = {r_erg:.4f} M")
print(f"  ISCO (prograde): r_ISCO = {r_isco:.4f} M")

## 2. Orbit Classification <a name="orbit-classification"></a>

Classify orbits based on their effective potential structure.

In [ ]:
# Test orbit classification
test_cases = [
    (1.2, 3.0, "Single thrust case"),
    (1.25, 3.1, "Continuous escape case"),
    (0.95, 2.5, "Bound orbit"),
    (1.5, 1.5, "Low angular momentum"),
]

print("Orbit Classification Results:")
print("="*60)

for E, Lz, desc in test_cases:
    props = classify_orbit(E, Lz, a, M)
    print(f"\n{desc}:")
    print(f"  E = {E:.2f}, Lz = {Lz:+.2f}")
    print(f"  Profile: {props.profile.name}")
    if props.r_periapsis:
        zone = "EXTRACTION" if props.in_extraction_zone else ("ERGOSPHERE" if props.in_ergosphere else "OUTSIDE")
        print(f"  Periapsis: {props.r_periapsis:.4f} M [{zone}]")

In [ ]:
# Effective potential visualization
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Case 1: Flyby orbit
plot_effective_potential(E=1.2, Lz=3.0, a=a, ax=axes[0])
axes[0].set_title('Flyby (E=1.2, Lz=3.0)')

# Case 2: Plunge orbit
plot_effective_potential(E=1.5, Lz=1.5, a=a, ax=axes[1])
axes[1].set_title('Plunge (E=1.5, Lz=1.5)')

plt.tight_layout()
plt.show()

## 3. Phase Space Visualization <a name="phase-space"></a>

Visualize orbit profiles across (E, Lz) parameter space.

In [ ]:
# Orbit profile map
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

plot_orbit_profile_map(
    a=a,
    E_range=(0.9, 2.0),
    Lz_range=(1.0, 5.0),
    n_E=60,
    n_Lz=60,
    ax=ax1
)

plot_periapsis_depth_map(
    a=a,
    E_range=(1.0, 2.0),
    Lz_range=(2.0, 5.0),
    n_E=60,
    n_Lz=60,
    ax=ax2
)

fig.suptitle(f'Parameter Space Analysis (a = {a})', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Spin dependence
radii = compute_key_radii_vs_spin(SPIN_VALUES)

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(radii['a'], radii['r_horizon'], 'k-', lw=2, label=r'$r_+$ (horizon)')
ax.plot(radii['a'], radii['r_ergosphere'], '--', color=COLORS['vermilion'], lw=2, label=r'$r_{\rm erg}$')
ax.plot(radii['a'], radii['r_isco_prograde'], '-.', color=COLORS['blue'], lw=2, label=r'$r_{\rm ISCO}$ (pro)')

ax.fill_between(radii['a'], radii['r_horizon'], radii['r_ergosphere'], 
                alpha=0.2, color=COLORS['vermilion'], label='Ergosphere')

ax.set_xlabel(r'Spin $a/M$')
ax.set_ylabel(r'Radius $r/M$')
ax.set_title('Key Radii vs Black Hole Spin')
ax.legend(loc='upper right')
ax.set_xlim(0.6, 1.0)
ax.set_ylim(0.5, 6)

plt.tight_layout()
plt.show()

## 4. Thrust Strategy Comparison <a name="strategy-comparison"></a>

Compare geodesic, single impulse, and burst thrust strategies.

In [ ]:
# Configure simulation
config = SimulationConfig(
    a=0.95,
    E0=1.2,
    Lz0=3.0,
    v_e=0.95,
    delta_m_fraction=0.2,
)

print(f"Simulation Configuration:")
print(f"  Initial energy: E0 = {config.E0}")
print(f"  Angular momentum: Lz0 = {config.Lz0}")
print(f"  Exhaust velocity: v_e = {config.v_e}c")
print(f"  Mass fraction: {config.delta_m_fraction*100}%")

In [ ]:
# Run strategy comparison
comparison = compare_strategies(
    config,
    strategies=[ThrustStrategy.NONE, ThrustStrategy.SINGLE_IMPULSE],
    verbose=True
)

In [ ]:
# Visualize single impulse trajectory
impulse_result = comparison.results.get(ThrustStrategy.SINGLE_IMPULSE)

if impulse_result and impulse_result.trajectory_data:
    data = impulse_result.trajectory_data
    
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    # Trajectory
    ax = axes[0]
    x = data['r'] * np.cos(data['phi'])
    y = data['r'] * np.sin(data['phi'])
    ax.plot(x, y, 'b-', lw=1)
    ax.add_patch(plt.Circle((0, 0), r_plus, color='black'))
    ax.add_patch(plt.Circle((0, 0), r_erg, fill=False, ls='--', color=COLORS['vermilion']))
    ax.set_xlabel(r'$x/M$')
    ax.set_ylabel(r'$y/M$')
    ax.set_aspect('equal')
    ax.set_title('Trajectory')
    
    # Radial evolution
    ax = axes[1]
    ax.plot(data['tau'], data['r'], 'b-')
    ax.axhline(r_erg, color=COLORS['vermilion'], ls='--', label=r'$r_{\rm erg}$')
    ax.axhline(r_plus, color='black', ls='-', label=r'$r_+$')
    ax.set_xlabel(r'$\tau/M$')
    ax.set_ylabel(r'$r/M$')
    ax.legend()
    ax.set_title('Radial Evolution')
    
    # Energy
    ax = axes[2]
    ax.plot(data['tau'], data['E'], 'g-')
    ax.axhline(config.E0, color='gray', ls=':', label=r'$E_0$')
    ax.set_xlabel(r'$\tau/M$')
    ax.set_ylabel(r'$E$')
    ax.legend()
    ax.set_title('Energy')
    
    fig.suptitle(f'Single Impulse: DeltaE = {impulse_result.Delta_E:+.4f}, E_ex = {impulse_result.E_ex_mean:.4f}')
    plt.tight_layout()
    plt.show()
else:
    print("No trajectory data available")

## 5. Ensemble Analysis <a name="ensemble-analysis"></a>

Statistical analysis over parameter distributions.

In [ ]:
# Configure ensemble
ensemble_config = EnsembleConfig(
    base_config=config,
    n_samples=50,  # Increase for production runs
    sampling_method='lhs',
    E_bounds=(1.15, 1.30),
    Lz_bounds=(2.8, 3.2),
    strategy=ThrustStrategy.SINGLE_IMPULSE,
    seed=42,
)

print("Ensemble configuration:")
print(f"  Samples: {ensemble_config.n_samples}")
print(f"  Method: {ensemble_config.sampling_method}")
print(f"  E bounds: {ensemble_config.E_bounds}")
print(f"  Lz bounds: {ensemble_config.Lz_bounds}")

In [ ]:
# Run ensemble
print("Running ensemble (this may take a moment)...")
ensemble_result = run_ensemble(ensemble_config, verbose=True)

In [ ]:
# Visualize ensemble results
df = ensemble_result.to_dataframe()
escaped = df[df['outcome'] == 'ESCAPE']

if len(escaped) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    # DeltaE histogram
    ax = axes[0]
    ax.hist(escaped['Delta_E'], bins=20, color=COLORS['green'], alpha=0.7)
    ax.axvline(escaped['Delta_E'].mean(), color='red', ls='--', label=f'Mean: {escaped["Delta_E"].mean():.4f}')
    ax.set_xlabel(r'$\Delta E$')
    ax.set_ylabel('Count')
    ax.legend()
    ax.set_title('Energy Gain Distribution')
    
    # E0 vs Lz0 colored by outcome
    ax = axes[1]
    colors = [COLORS['green'] if o == 'ESCAPE' else COLORS['vermilion'] for o in df['outcome']]
    ax.scatter(df['E0'], df['Lz0'], c=colors, s=30, alpha=0.6)
    ax.set_xlabel(r'$E_0$')
    ax.set_ylabel(r'$L_z$')
    ax.set_title('Outcome Map')
    
    # DeltaE vs r_min
    ax = axes[2]
    ax.scatter(escaped['r_min'], escaped['Delta_E'], c=COLORS['blue'], s=30, alpha=0.6)
    ax.set_xlabel(r'$r_{\rm min}/M$')
    ax.set_ylabel(r'$\Delta E$')
    ax.set_title('Energy Gain vs Periapsis')
    
    plt.tight_layout()
    plt.show()
else:
    print("No escaped trajectories in ensemble")

In [ ]:
# Find optimal configurations
sweet_spot = find_sweet_spot(ensemble_result, n_top=5)

print("Top 5 configurations by energy gain:")
print("="*60)
for i, cfg in enumerate(sweet_spot):
    print(f"{i+1}. E0 = {cfg['E0']:.4f}, Lz0 = {cfg['Lz0']:.4f}")
    print(f"   DeltaE = {cfg['Delta_E']:+.4f}, eta_cum = {cfg['eta_cumulative']*100:.2f}%")

## 6. Interactive Parameter Exploration <a name="interactive"></a>

Explore parameters interactively (requires ipywidgets).

In [ ]:
# Interactive exploration function
def explore_orbit(E, Lz, a_spin):
    """Explore orbit properties interactively."""
    props = classify_orbit(E, Lz, a_spin)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    # Effective potential
    plot_effective_potential(E, Lz, a_spin, ax=ax1)
    
    # Info panel
    ax2.axis('off')
    info = [
        f"Energy: E = {E:.3f}",
        f"Angular momentum: Lz = {Lz:.3f}",
        f"Spin: a/M = {a_spin:.3f}",
        "",
        f"Profile: {props.profile.name}",
        f"Periapsis: {props.r_periapsis:.4f} M" if props.r_periapsis else "Periapsis: None",
        f"In ergosphere: {props.in_ergosphere}",
        f"In extraction zone: {props.in_extraction_zone}",
        "",
        f"Horizon: {horizon_radius(a_spin):.4f} M",
        f"Ergosphere: {ergosphere_radius(np.pi/2, a_spin):.4f} M",
    ]
    ax2.text(0.1, 0.9, "\n".join(info), transform=ax2.transAxes, 
             fontsize=11, verticalalignment='top', family='monospace')
    ax2.set_title('Orbit Properties')
    
    plt.tight_layout()
    plt.show()

# Example call
explore_orbit(E=1.2, Lz=3.0, a_spin=0.95)

In [ ]:
# Try different parameters
explore_orbit(E=1.1, Lz=2.5, a_spin=0.99)

---

## Summary

This notebook provides interactive tools for:

1. **Orbit Classification**: Understanding which (E, Lz) combinations lead to different trajectory types
2. **Phase Space Maps**: Visualizing the parameter space structure
3. **Strategy Comparison**: Evaluating single impulse vs geodesic performance
4. **Ensemble Analysis**: Statistical characterization of extraction efficiency

For production runs, use the command-line script:
```bash
python experiments/run_trajectory_study.py --mode standard
```